<a href="https://colab.research.google.com/github/rajagopal17/Text-Analysis-with-Spacy/blob/master/Glove_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', size=14)
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import csv
import re
from __future__ import unicode_literals
import spacy
from spacy.tokens import doc
nlp=spacy.load('en')
import en_core_web_sm
#nlp=en_core_web_md.load()
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
parser = English()
import string
punctuations=string.punctuation
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

import gensim,pprint
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
from gensim.models import Phrases
#from gensim.models.word2vec import sentences
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import corpora, models,similarities
from gensim.parsing.preprocessing import preprocess_documents
from gensim.models import Word2Vec, KeyedVectors 

In [ ]:
!git clone https://github.com/MohammadWasil/Sentiment-Analysis-IMDb-Movie-Review.git

Cloning into 'Sentiment-Analysis-IMDb-Movie-Review'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70
Unpacking objects: 100% (70/70), done.


In [ ]:
data_train =pd.read_csv('/content/Sentiment-Analysis-IMDb-Movie-Review/labeledTrainData.tsv', delimiter='\t',encoding="utf-8")
#data_test =('/content/Sentiment-Analysis-IMDb-Movie-Review/testData.tsv')
data_file = data_train[['sentiment','review']].copy()
train_ds = data_file.head(20000).copy()
test_ds  = data_file.tail(5000).copy()
train_ds.to_csv('/content/train_ds.csv',index=False)
test_ds.to_csv('/content/test_ds.csv',index=False)
train_ds.head()
#data_file.describe

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


# Convert column to string, apply nlp pipe lines for lemmatization,ents, etc
  

1.   Convert column to list and feed it back to new data frame
2.   Get frequency and plot on graph



In [ ]:
#Convert each row of reviews column to string and store it in a file

temp_file=data_train['Reviews'].apply(lambda x:str(x))
temp_file=data_train['Reviews'].apply(lambda x:re.sub('[^A-Z 0-9 a-z-]+','',x))
temp_file

In [ ]:
lemma_list=list([token.lemma_ for token in doc if token.is_stop==False] for doc in nlp.pipe(temp_file, n_threads=2,batch_size=1000,disable=['tagger','parser','ner']))


In [ ]:
lemma_names=list([token.text for token in doc if token.pos_=='PROPN'] for doc in nlp.pipe(temp_file,batch_size=1000))

In [ ]:
lemma_person=list([token.text for token in doc if token.ent_type_=='PERSON'] for doc in nlp.pipe(temp_file,batch_size=1000))

In [ ]:
get_names=[]
for x in lemma_person:
  for y in x:
    get_names.append(y)
get_names

In [ ]:
data_clean['lemma']=lemma_list
data_clean['Reviews']=lemma_names
data_clean['names']=lemma_person

In [ ]:
data_clean.head(20)

In [ ]:
from collections import Counter
word_freq=Counter(get_names)
word_freq



In [ ]:
word_freq_graph =pd.DataFrame(list(word_freq.items()),columns=['name','freq'])
word_freq_graph
final_df=word_freq_graph[word_freq_graph['freq']>800 ]
final_df

In [ ]:
import matplotlib.pyplot as plot
final_df.plot.barh(x='name', y='freq', title="Frequency of the mention of lead character's name");

plot.show(block=True);

# Torch Text

In [ ]:
from torchtext.data import Field,TabularDataset,BucketIterator

In [ ]:
#X_train,X_test,y_train,y_test = train_test_split('/content/Sentiment-Analysis-IMDb-Movie-Review/labeledTrainData.tsv')

In [ ]:
!git clone https://github.com/AladdinPerzon/Machine-Learning-Collection.git

Cloning into 'Machine-Learning-Collection'...
remote: Enumerating objects: 9466, done.
remote: Total 9466 (delta 0), reused 0 (delta 0), pack-reused 9466
Receiving objects: 100% (9466/9466), 1.07 GiB | 39.99 MiB/s, done.
Resolving deltas: 100% (661/661), done.
Checking out files: 100% (154/154), done.


In [ ]:
def tokenize(text):
  return[token.text for token in nlp(text)]

In [ ]:
#tokenize= lambda x: x.split()
TEXT  = Field(sequential= True,use_vocab=True,tokenize=tokenize,lower=True)
LABEL = Field(sequential=False,use_vocab=False)
fields={'sentiment':LABEL,'review':TEXT}

#Split dataset for train & test

In [ ]:
#train_data, test_data= TabularDataset(path='/content/sample_data/test_d.csv',format='csv',fields={'sentiment':sentiment,'review':review})
train = TabularDataset(path='/content/sample_data/train_ds.csv', 
                        format='csv', 
                        fields=[("sentiment",LABEL),
                                ("review",TEXT)],  
                        skip_header=True)
test = TabularDataset(path='/content/sample_data/test_ds.csv', 
                        format='csv', 
                        fields= [("sentiment",LABEL),
                                ("review",TEXT)],  
                        skip_header=True)                             
                       

In [ ]:
TEXT.build_vocab(train,max_size=10000,min_freq=2)

In [ ]:
TEXT.vocab.itos

#TEXT.vocab.stoi['movie']

In [ ]:
train_ds.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


# Split data for train and test

In [ ]:
X=train_ds[['review']].copy()
y=train_ds[['sentiment']].copy()

Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,train_size=0.7,random_state=32,shuffle=True)

# Convert reviews columns to list of tokens to feed into Glove, W2V algorithms

In [ ]:
lines=Xtest['review'].apply(lambda x: str(x))

lines_list=list([token.lemma_.lower() for token in doc if token.is_alpha] 
                for doc in nlp.pipe(lines, batch_size=1000,disable=['tagger','parser','ner']))


14708    This is what I was expecting when star trek DS...
14706    I´m glad that someone has made a movie about h...
15628    David Lynch's (1999) film of John Roach / Mary...
16510    When I first saw this movie, I thought it was ...
18155    I didn't expect much when I first saw the DVD ...
                               ...                        
18340    It Could Have Been A Marvelous Story Based On ...
10066    This is one of the finest films to come out of...
2577     This film never received the attention it dese...
8821     Guys and Dolls has to be one of my favorite mu...
8469     Creature Comforts in America should have been ...
Name: review, Length: 6000, dtype: object

In [ ]:
lines_list[0]

# How to use standard Glove vectors for finding similarities without Gensim

To load the pre-trained vectors, we must first create a dictionary that will hold the mappings between words, and the embedding vectors of those words.

In [ ]:
import os
import numpy as np
GLOVE_DIR ='/content/drive/My Drive/Python'
print('Indexing word vectors.')
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

#print(embeddings_index['banana'])

Indexing word vectors.
Found 400001 word vectors.


In [ ]:
from scipy import spatial
def find_closest_embeddings(embedding):
    return sorted(embeddings_index.keys(), key=lambda word: spatial.distance.euclidean(embeddings_index[word], embedding))


In [ ]:
find_closest_embeddings(embeddings_index["role"])[:20]

['role',
 'roles',
 'as',
 'notably',
 'involved',
 'addition',
 'supporting',
 'pivotal',
 'besides',
 'also',
 'portrayed',
 'part',
 'character',
 'both',
 'important',
 'well',
 'particular',
 'involvement',
 'fact',
 'although']

# Converting  Standard Glove vectors into Gensim- Word2Vec format for finding similarities

In [ ]:
!pip3 install glove_python
#https://medium.com/analytics-vidhya/word-vectorization-using-glove-76919685ee0b
#https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db

     |████████████████████████████████| 266kB 2.7MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700259 sha256=6eb47624cfd59ea97fc90716af4d4b13ae5520183d5288b9d5b41695de455110
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_input_file='/content/drive/My Drive/Python/glove.6B.300d.txt', word2vec_output_file="/content/drive/My Drive/Python/gensim_glove_vectors.txt")    
from gensim.models.keyedvectors import KeyedVectors
model = KeyedVectors.load_word2vec_format("/content/drive/My Drive/Python/gensim_glove_vectors.txt", binary=False)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-09 06:23:07,336 : INFO : converting 400001 vectors from /content/drive/My Drive/Python/glove.6B.300d.txt to /content/drive/My Drive/Python/gensim_glove_vectors.txt
2020-08-09 06:23:26,344 : INFO : loading projection weights from /content/drive/My Drive/Python/gensim_glove_vectors.txt
2020-08-09 06:25:28,131 : INFO : loaded (400001, 300) matrix from /content/drive/My Drive/Python/gensim_glove_vectors.txt


In [ ]:

model.most_similar('lead')



/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('leads', 0.6556434631347656),
 ('leading', 0.6076206564903259),
 ('led', 0.5886493921279907),
 ('second', 0.5558751225471497),
 ('advantage', 0.5509870052337646),
 ('third', 0.5497407913208008),
 ('victory', 0.547242283821106),
 ('ahead', 0.5369311571121216),
 ('win', 0.5189648270606995),
 ('take', 0.5123140811920166)]

# Train Glove model on own corpus, find similarities and save the model

https://github.com/alexeygrigorev/avito-duplicates-kaggle/blob/master/prepare_glove_model.py

In [ ]:
from time import time
from glove import Glove, Corpus

def train_glove(sentences):
    print ('training glove model...')
    t0 = time()
    
    num_features = 300    # Word vector dimensionality
    context = 5          # Context window size
    learning_rate = 0.05
    
    corpus = Corpus()
    corpus.fit(sentences, window=context)

    glove = Glove(no_components=num_features, learning_rate=learning_rate)
    glove.fit(corpus.matrix, epochs=30, no_threads=8, verbose=True)
    glove.add_dictionary(corpus.dictionary)

    print('took %0.5fs.' % (time() - t0))
    return (glove)

In [ ]:

glove_model=train_glove(lines_list)

training glove model...
Performing 30 training epochs with 8 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
took 198.76000s.


In [ ]:
glove_model.most_similar('lead',number=6)

[('female', 0.8574840795906655),
 ('lady', 0.8195846557046927),
 ('male', 0.8016110046123489),
 ('portray', 0.7655702525889665),
 ('actress', 0.7363168113668669)]

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
glove_model.save('/content/drive/My Drive/Python/glove_final.model')


In [ ]:
mmodel=glove_model.load('/content/drive/My Drive/Python/glove_final.model')

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
#from gensim.models import Word2Vec, KeyedVectors 
#rmodel=model.load('/content/drive/My Drive/Python/glove_final.model')
glove_model.save('/content/drive/My Drive/Python/glove_model.txt')

In [ ]:

from gensim.models.word2vec import Word2Vec
from gensim import utils
from glove import Glove
#from most_similar import most_similar_dist
#rmodel.most_similar('role',number=6)
#gl_model=model.load('/content/drive/My Drive/Python/glove_model.txt')
glve = utils.unpickle('/content/drive/My Drive/Python/glove_final.model')
#most_similar_dist('cast',topn=20)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
glve.items()

dict_items([('no_components', 300), ('learning_rate', 0.05), ('alpha', 0.75), ('max_count', 100.0), ('max_loss', 10.0), ('word_vectors', array([[ 0.1379273 ,  0.1523837 , -0.03909691, ...,  0.0023002 ,
        -0.05702711, -0.01808888],
       [-0.04127934,  0.20287251, -0.25291654, ..., -0.085623  ,
        -0.25946611,  0.06567444],
       [-0.13196684,  0.01707219, -0.05161704, ..., -0.0125902 ,
        -0.20142275,  0.11007754],
       ...,
       [ 0.01503953, -0.01742775,  0.01461459, ..., -0.0174191 ,
         0.00850276,  0.01982994],
       [ 0.00427301, -0.0165074 ,  0.01436856, ..., -0.00539025,
         0.00832229,  0.00053936],
       [ 0.0028445 , -0.01196818,  0.01182349, ...,  0.00053011,
         0.01191865,  0.00545855]])), ('word_biases', array([ 0.43429262,  0.96325347, -0.26294471, ..., -0.1633423 ,
       -0.13968808, -0.06532717])), ('vectors_sum_gradients', array([[ 26.81652324,  37.96447346,  31.46666894, ...,  38.74990222,
         30.35622909,  46.35581638],


# Embedding using Gensim-Word2Vec on own corpus
  Save the model and reuse the saved model

In [ ]:
w2v_model=gensim.models.Word2Vec(lines_list,size=50,cbow_mean=5,min_count=1,workers=3)

2020-08-09 06:38:45,506 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2020-08-09 06:38:45,509 : INFO : collecting all words and their counts
2020-08-09 06:38:45,510 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-09 06:38:45,792 : INFO : collected 32401 word types from a corpus of 1352208 raw words and 6000 sentences
2020-08-09 06:38:45,794 : INFO : Loading a fresh vocabulary
2020-08-09 06:38:45,860 : INFO : effective_min_count=1 retains 32401 unique words (100% of original 32401, drops 0)
2020-08-09 06:38:45,861 : INFO : effective_min_count=1 leaves 1352208 word corpus (100% of original 1352208, drops 0)
2020-08-09 06:38:45,980 : INFO : deleting the raw counts dictionary of 32401 items
2020-08-09 06:38:45,982 : INFO : sample=0.001 downsamples 49 most-common words
2020-08-09 06:38:45,984 : INFO : downsampling leaves estimated 964777 word corpus (71.3% of prior 1352208)
2020-08-09 06:38:46,099 : INFO : estimated req

In [ ]:
w2v_model.train(lines_list, total_examples=len(lines_list), epochs=30)

2020-08-09 06:39:18,189 : WARNING : Effective 'alpha' higher than previous training cycles
2020-08-09 06:39:18,190 : INFO : training model with 3 workers on 32401 vocabulary and 50 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2020-08-09 06:39:19,213 : INFO : EPOCH 1 - PROGRESS: at 49.08% examples, 473197 words/s, in_qsize 5, out_qsize 0
2020-08-09 06:39:20,217 : INFO : EPOCH 1 - PROGRESS: at 97.52% examples, 466902 words/s, in_qsize 3, out_qsize 1
2020-08-09 06:39:20,219 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-09 06:39:20,231 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-09 06:39:20,250 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-09 06:39:20,251 : INFO : EPOCH - 1 : training on 1352208 raw words (965416 effective words) took 2.0s, 471451 effective words/s
2020-08-09 06:39:21,268 : INFO : EPOCH 2 - PROGRESS: at 49.87% examples, 479592 words/s, in_qsize 5, out_qsize 0
2020-08-09 0

(28941759, 40566240)

In [ ]:
w2v_model.wv.most_similar('role')

2020-08-09 06:41:06,090 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('performance', 0.6427462100982666),
 ('appearance', 0.6387189626693726),
 ('actress', 0.6110916137695312),
 ('actor', 0.6104315519332886),
 ('part', 0.5892289876937866),
 ('cast', 0.5829398036003113),
 ('career', 0.5704333186149597),
 ('presence', 0.5526494979858398),
 ('performer', 0.5471001863479614),
 ('persona', 0.5357048511505127)]

In [ ]:

w2v_model.save("/content/drive/My Drive/Python/word2vec.model")

2020-08-08 16:10:57,482 : INFO : saving Word2Vec object under /content/drive/My Drive/Python/word2vec.model, separately None
2020-08-08 16:10:57,485 : INFO : not storing attribute vectors_norm
2020-08-08 16:10:57,487 : INFO : not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-08 16:10:57,888 : INFO : saved /content/drive/My Drive/Python/word2vec.model


In [ ]:
#from gensim.models import Word2Vec, KeyedVectors 

loaded_model = Word2Vec.load('/content/drive/My Drive/Python/word2vec.model')

2020-08-08 16:11:00,568 : INFO : loading Word2Vec object from /content/drive/My Drive/Python/word2vec.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-08 16:11:00,768 : INFO : loading wv recursively from /content/drive/My Drive/Python/word2vec.model.wv.* with mmap=None
2020-08-08 16:11:00,769 : INFO : setting ignored attribute vectors_norm to None
2020-08-08 16:11:00,770 : INFO : loading vocabulary recursively from /content/drive/My Drive/Python/word2vec.model.vocabulary.* with mmap=None
2020-08-08 16:11:00,773 : INFO : loading trainables recursively from /content/drive/My Drive/Python/word2vec.model.trainables.* with mmap=None
2020-08-08 16:11:00,775 : INFO : setting igno

In [ ]:
loaded_model.wv.most_similar('role')

2020-08-08 16:11:05,149 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('actor', 0.6544277667999268),
 ('actress', 0.6178332567214966),
 ('cast', 0.6073322892189026),
 ('part', 0.596723735332489),
 ('appearance', 0.5944481492042542),
 ('performance', 0.5907403826713562),
 ('presence', 0.5749421119689941),
 ('charisma', 0.5361968874931335),
 ('persona', 0.5223928689956665),
 ('character', 0.5197198987007141)]

In [ ]:
sim_words = {words: [item[0] for item in loaded_model.wv.most_similar([words], topn=5)]
                  for words in ['plot', 'movie','play','role','cast','story','history']}

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
sim_words
#sim_words2

{'cast': ['actor', 'ensemble', 'actress', 'role', 'act'],
 'history': ['epic', 'literature', 'era', 'cinema', 'importance'],
 'movie': ['film', 'flick', 'documentary', 'it', 'sequel'],
 'play': ['perform', 'portray', 'conanesque', 'isla', 'meet'],
 'plot': ['storyline', 'story', 'script', 'scenario', 'premise'],
 'role': ['actor', 'actress', 'cast', 'part', 'appearance'],
 'story': ['plot', 'storyline', 'tale', 'premise', 'script']}

In [ ]:
loaded_model.wv.vocab.keys()

dict_keys(['this', 'be', 'what', 'i', 'expect', 'when', 'star', 'trek', 'premiere', 'not', 'to', 'slight', 'that', 'a', 'wonderful', 'show', 'in', '-pron-', 'own', 'right', 'however', 'it', 'never', 'really', 'give', 'the', 'fan', 'much', 'of', 'they', 'want', 'enterprise', 'while', 'have', 'similarity', 'original', 'differ', 'enough', 'way', 'make', 'idea', 'exploration', 'excite', 'us', 'again', 'and', 'one', 'primary', 'ingredient', 'so', 'love', 'another', 'success', 'relationship', 'evolve', 'between', 'crow', 'member', 'viewers', 'care', 'deeply', 'for', 'promise', 'area', 'good', 'chemistry', 'bakula', 'blalock', 'seem', 'very', 'sexual', 'tension', 'can', 'often', 'become', 'crutch', 'feel', 'on', 'lead', 'say', 'alot', 'than', 'typical', 'think', 'we', 'deal', 'with', 'such', 'grand', 'scale', 'character', 'different', 'race', 'or', 'species', 'even', 'get', 'some', 'interest', 'television', 'also', 'should', 'note', 'performance', 'convince', 'vulcan', 'bacula', 'whimsy', 'st

In [ ]:
#a=['film', 'plot', 'movie','play','role','cast','story','history']
j=KeyedVectors.load('/content/drive/My Drive/Python/word2vec.model')

print( [x[0] for x in j.most_similar([v])] for v in ['film', 'plot', 'movie','play','role','cast','story','history'])

2020-08-08 16:11:50,227 : INFO : loading Word2VecKeyedVectors object from /content/drive/My Drive/Python/word2vec.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-08-08 16:11:50,684 : INFO : loading wv recursively from /content/drive/My Drive/Python/word2vec.model.wv.* with mmap=None
2020-08-08 16:11:50,686 : INFO : setting ignored attribute vectors_norm to None
2020-08-08 16:11:50,686 : INFO : loading vocabulary recursively from /content/drive/My Drive/Python/word2vec.model.vocabulary.* with mmap=None
2020-08-08 16:11:50,690 : INFO : loading trainables recursively from /content/drive/My Drive/Python/word2vec.model.trainables.* with mmap=None
2020-08-08 16:11:50,694 : INFO : 

<generator object <genexpr> at 0x7f4822fe4258>


In [ ]:
# Get a list of words in the vocabulary
words = loaded_model.wv.vocab.keys()

# Make a dictionary
we_dict = {word:loaded_model.wv.most_similar([word]) for word in words}

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print([ x[0] for x in j.wv.most_similar('story')])

2020-08-08 16:13:54,019 : INFO : precomputing L2-norms of word weight vectors


['plot', 'storyline', 'tale', 'premise', 'script', 'concept', 'truth', 'drama', 'documentary', 'narrative']


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
sim_words2 = {words: [item[0] for item in mmodel.most_similar(words)]
                  for words in ['plot', 'movie','play','role','cast','story','history']}
sim_words2

{'cast': ['support', 'excellent', 'actor', 'member'],
 'history': ['cinema', 'future', 'genre', 'america'],
 'movie': ['b', 'suck', 'horror', 'br'],
 'play': ['actress', 'portray', 'role', 'actor'],
 'plot': ['twist', 'hole', 'device', 'weak'],
 'role': ['play', 'portray', 'lead', 'title'],
 'story': ['line', 'true', 'ghost', 'hodges']}

In [ ]:
sim_words

{'cast': ['actor', 'ensemble', 'actress', 'role', 'act'],
 'history': ['epic', 'literature', 'era', 'cinema', 'importance'],
 'movie': ['film', 'flick', 'documentary', 'it', 'sequel'],
 'play': ['perform', 'portray', 'conanesque', 'isla', 'meet'],
 'plot': ['storyline', 'story', 'script', 'scenario', 'premise'],
 'role': ['actor', 'actress', 'cast', 'part', 'appearance'],
 'story': ['plot', 'storyline', 'tale', 'premise', 'script']}

# Word embeddings using Fasttext
https://stackabuse.com/python-for-nlp-working-with-facebook-fasttext-library/

In [ ]:
from gensim.models.fasttext import FastText

In [ ]:
embedding_size = 60
window_size = 40
min_word = 5
down_sampling = 1e-2

In [ ]:
%%time
ft_model = FastText(lines_list,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                      sg=1,
                      iter=30)

In [ ]:
ft_model.wv.most_similar('castin')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('casting', 0.9168997406959534),
 ('excellent', 0.7221461534500122),
 ('brilliant', 0.7150087356567383),
 ('actor', 0.7068393230438232),
 ('performance', 0.7058931589126587),
 ('performances', 0.698100209236145),
 ('excellently', 0.6922032833099365),
 ('cast', 0.6892410516738892),
 ('superb', 0.6729656457901001),
 ('brilliantly', 0.6714730262756348)]